In [1]:
# define hp model
from source_codes.ranking_models_hptunable import comparison_ranking_model_hp,ranking_alone_model_hp
from keras import regularizers

def build_model(hp):
    hp_model = ranking_alone_model_hp(
        # feature extraction hyperparameters
        backbone='vgg19',
        backbone_weights='imagenet',

        # subnetwork hyperparameters
        r_num_dense=hp.Int('r_num_dense',max_value=4,min_value=2,step=1,default=2),
        r_dense_dropout=True,
        r_dense_units=hp.Int('r_dense_units',max_value=4096,min_value=512,step=512,default=1024),
        r_sub_dense_mode=hp.Choice('r_sub_dense_mode',['parallel','decreasing'],default='decreasing'),
        r_dense_dropout_rate=hp.Float(name='r_dense_dropout_rate',max_value=0.5,min_value=0.1,step=0.1),
        r_l1 = hp.Float('r_l1', max_value=1e-4, min_value=1e-6, step=10,sampling='log',default=1e-5),
        r_l2  = hp.Float('r_l2', max_value=1e-4, min_value=1e-6, step=10,sampling='log',default=1e-5),

        # compilation hyperparameters
        optimizer=hp.Choice('optimizer',['sgd','adam'],default='sgd'),
        decay=hp.Float('decay', max_value=1e-6, min_value=1e-8, step=10,sampling='log',default=1e-7),
        learning_rate=hp.Float('learning_rate', max_value=1e-6, min_value=1e-8, step=10,sampling='log',default=1e-7),
    )
    return hp_model


In [2]:
# configure hp model
import keras_tuner as kt

build_model(kt.HyperParameters())

val_rank_accuracy_obj = kt.Objective("val_ranking_accuracy", direction="max")

tuner = kt.BayesianOptimization(
    build_model,
    objective=[val_rank_accuracy_obj],
    max_trials=25,
    executions_per_trial=4,
    overwrite=False,
    directory='model_training/Q1/hp_tuning',
    project_name="hptuning_ranking_230430",
)

In [3]:
# prepare data
import numpy as np
import os

question_index = str(1)

# training data
x_left_training = np.load(os.path.join('data/datasets/train_data/draw_exclude/train_left_duel_' + question_index + '.npy'), allow_pickle=True)
x_right_training = np.load(os.path.join('data/datasets/train_data/draw_exclude/train_right_duel_' + question_index + '.npy'), allow_pickle=True)
y_training = np.load(os.path.join('data/datasets/train_data/draw_exclude/train_label_duel_' + question_index + '.npy'), allow_pickle=True)

# validation data
x_left_val = np.load(os.path.join('data/datasets/validation_data/draw_exclude/val_left_duel_' + question_index + '.npy'), allow_pickle=True)
x_right_val = np.load(os.path.join('data/datasets/validation_data/draw_exclude/val_right_duel_' + question_index + '.npy'), allow_pickle=True)
y_val = np.load(os.path.join('data/datasets/validation_data/draw_exclude/val_label_duel_' + question_index + '.npy'), allow_pickle=True)

# test data
x_left_test = np.load(os.path.join('data/datasets/test_data/draw_exclude/test_left_duel_' + question_index + '.npy'), allow_pickle=True)
x_right_test = np.load(os.path.join('data/datasets/test_data/draw_exclude/test_right_duel_' + question_index + '.npy'), allow_pickle=True)
y_test = np.load(os.path.join('data/datasets/test_data/draw_exclude/test_label_duel_' + question_index + '.npy'), allow_pickle=True)

In [4]:
# search hp model
from keras import callbacks
early_stopping = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=15,
    mode='min',
    restore_best_weights=True
)

tuner.search(
    x=[x_left_training, x_right_training],
    y=y_training,
    epochs=80,
    batch_size=16,
    validation_data = ([x_left_val,x_right_val],y_val),
    callbacks=[early_stopping]
)

Trial 25 Complete [01h 29m 55s]
multi_objective: -0.7467672228813171

Best multi_objective So Far: -0.7860991358757019
Total elapsed time: 1d 11h 11m 16s
INFO:tensorflow:Oracle triggered exit


In [5]:
# Get the top 10 hyperparameters results
tuner.results_summary()

Results summary
Results in model_training/Q1/hp_tuning\hptuning_ranking_230430
Showing 10 best trials
Trial summary
Hyperparameters:
r_num_dense: 3
r_dense_units: 1024
r_sub_dense_mode: decreasing
r_dense_dropout_rate: 0.1
r_l1: 9.999999999999999e-05
r_l2: 1e-06
optimizer: adam
decay: 1e-07
learning_rate: 1e-06
Score: -0.7860991358757019
Trial summary
Hyperparameters:
r_num_dense: 4
r_dense_units: 3072
r_sub_dense_mode: decreasing
r_dense_dropout_rate: 0.1
r_l1: 9.999999999999999e-06
r_l2: 1e-06
optimizer: adam
decay: 1e-07
learning_rate: 1e-07
Score: -0.7844827771186829
Trial summary
Hyperparameters:
r_num_dense: 4
r_dense_units: 3072
r_sub_dense_mode: parallel
r_dense_dropout_rate: 0.2
r_l1: 9.999999999999999e-05
r_l2: 9.999999999999999e-06
optimizer: adam
decay: 1e-08
learning_rate: 1e-07
Score: -0.7796336114406586
Trial summary
Hyperparameters:
r_num_dense: 2
r_dense_units: 3072
r_sub_dense_mode: parallel
r_dense_dropout_rate: 0.30000000000000004
r_l1: 9.999999999999999e-05
r_l2: 1

In [6]:
# Get the top 5 weighted models.
models = tuner.get_best_models(num_models=5)
best_model = models[0]
best_model.summary()

# Save the best 5 models initialized with the best weights.
save_dir = 'model_training/Q1/hp_tuning/hptuning_ranking_230430'
for i in range(len(models)):
    models[i].save(os.path.join(save_dir, 'best_model_' + str(i) + '.h5'))

Model: "Ranking_Model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 vgg19 (Functional)             (None, 7, 7, 512)    20024384    ['input_2[0][0]',                
                                                                  'input_3[0][0]']    

In [7]:
# save the best model architecture
best_hps = tuner.get_best_hyperparameters(5)
save_dir = "model_training/Q1/hp_tuning/hptuning_ranking_230430"

for i in range(len(best_hps)):
    model = build_model(best_hps[i])
    model.save(os.path.join(save_dir, 'best_model_' + str(i) + '_architecture.h5'))

In [9]:
# Retrain the top n model
model = build_model(best_hps[0])
save_model_results = os.path.join(save_dir, "bestmodel0_retrain_results")
checkpoint = callbacks.ModelCheckpoint(
    os.path.join(save_model_results,'best_performance_checkpoint.h5'),
    save_best_only=True,
    save_weights_only=True,
    monitor="val_loss",
    mode='min',
    verbose=1
)

early_stopping = callbacks.EarlyStopping(monitor="val_loss",
                                         patience=10,
                                         mode='min',
                                         verbose=1)
model.summary()

Model: "Ranking_Model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_20 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_21 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 vgg19 (Functional)             (None, 7, 7, 512)    20024384    ['input_20[0][0]',               
                                                                  'input_21[0][0]']   

In [14]:
# train the model
history = model.fit(
    x=[x_left_training, x_right_training],
    y=y_training,
    epochs=50,
    batch_size=16,
    validation_data = ([x_left_val,x_right_val],y_val),
    callbacks=[early_stopping, checkpoint]
)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [13]:
from keras import backend
backend.clear_session()

In [1]:
# Save the model history
# Show performance plots
import pickle
import os
from source_codes.utils import plot_model_accuracy, plot_model_loss, save_structure_in_json,plot_model_metrics
import matplotlib.pyplot as plt

folder_path = save_model_results
model_history = history

# Save training history
with open(os.path.join(folder_path, 'history.pickle'), 'wb') as f:
    pickle.dump(model_history.history, f)

# Comparison Accuracy
com_acc = model.history.history['Comparison_Output_accuracy']
val_com_acc = model.history.history['val_Comparison_Output_accuracy']
epochs = range(1, len(com_acc) + 1)

# Configuring plot
plt.figure()
plt.plot(epochs, com_acc, 'bo', label='Training acc')
plt.plot(epochs, val_com_acc, 'b', label='Validation acc')
plt.title('Comparison Training and validation accuracy')
plt.legend()
plt.savefig(os.path.join(folder_path,'comparison_acc.png'))

# Comparison Loss
com_loss = model.history.history['Comparison_Output_loss']
val_com_loss = model.history.history['val_Comparison_Output_loss']
epochs = range(1, len(com_loss) + 1)

# Configuring plot
plt.figure()
plt.plot(epochs, com_loss, 'bo', label='Training loss')
plt.plot(epochs, val_com_loss, 'b', label='Validation loss')
plt.title('Comparison Training and validation loss')
plt.legend()
plt.savefig(os.path.join(folder_path,'comparison_loss.png'))

# Ranking Accuracy
rank_acc = model.history.history['Ranking_Score_Pair_accuracy']
val_rank_acc = model.history.history['val_Ranking_Score_Pair_accuracy']
epochs = range(1, len(rank_acc) + 1)

# Configuring plot
plt.figure()
plt.plot(epochs, rank_acc, 'bo', label='Training acc')
plt.plot(epochs, val_rank_acc, 'b', label='Validation acc')
plt.title('Ranking Training and validation accuracy')
plt.legend()
plt.savefig(os.path.join(folder_path,'ranking_acc.png'))

# Ranking Loss
rank_loss = model.history.history['Ranking_Score_Pair_loss']
val_rank_loss = model.history.history['val_Ranking_Score_Pair_loss']
epochs = range(1, len(rank_loss) + 1)

# Configuring plot
plt.figure()
plt.plot(epochs, rank_loss, 'bo', label='Training loss')
plt.plot(epochs, val_rank_loss, 'b', label='Validation loss')
plt.title('Ranking Training and validation loss')
plt.legend()
plt.savefig(os.path.join(folder_path,'ranking_loss.png'))

NameError: name 'save_model_results' is not defined